In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
# outputs = model(**inputs, labels=labels)
# loss = outputs.loss
# logits = outputs.logits

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
jsonObj = pd.read_json(path_or_buf='train.jsonl', lines=True)

jsonObj.info()
print(jsonObj['sentence1'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   word       5000 non-null   object 
 1   sentence1  5000 non-null   object 
 2   sentence2  5000 non-null   object 
 3   idx        5000 non-null   int64  
 4   label      5000 non-null   bool   
 5   start1     5000 non-null   int64  
 6   start2     5000 non-null   int64  
 7   end1       5000 non-null   int64  
 8   end2       5000 non-null   int64  
 9   version    5000 non-null   float64
dtypes: bool(1), float64(1), int64(5), object(3)
memory usage: 356.6+ KB
0             Do you want to come over to my place later?
1                                        Approach a task.
2                                              Run rogue.
3       The general ordered the colonel to hold his po...
4                 We like to summer in the Mediterranean.
                              ...                 

In [21]:
#Helper functions
def find_word_in_tokenized_sentence(word,token_ids):
    decomposedWord = tokenizer(word)
    # Iterate through to find a matching sublist of the token_ids
    for i in range(len(token_ids)):
        if token_ids[i] == decomposedWord[0] and token_ids[i:i+len(decomposedWord)] == decomposedWord:
            return (i,i+len(decomposedWord)-1)
    # This is the ouput when no matching pattern is found
    return (-1,-1)
  
def find_words_in_tokenized_sentences(wordList,token_ids):
    intList = []
    for word in wordList:
        if len(intList) == 0:
            intList.append(find_word_in_tokenized_sentence(word,token_ids))
        else:
            afterLastInterval = intList[-1][1]+1
            interv = find_word_in_tokenized_sentence(word,token_ids[afterLastInterval:])
            actualPositions = (interv[0] + afterLastInterval,interv[1]+afterLastInterval)
            intList.append(actualPositions)
    return intList

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels, return_predict_correctness = False):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    if return_predict_correctness:
        return np.sum(pred_flat == labels_flat) / len(labels_flat), pred_flat == labels_flat
    else:
        return np.sum(pred_flat == labels_flat) / len(labels_flat)

def flat_predictions(preds):
    pred_flat = np.argmax(preds, axis=1).flatten()
    return pred_flat == 1

In [32]:
#Pre process Data for training here:
batch_words = jsonObj['word'].astype(str).values.tolist()
batch_idx = jsonObj['idx'].astype(str).values.tolist()
batch_label = jsonObj['label'].astype(str).values.tolist()

batch_sentences = jsonObj['sentence1'].astype(str).values.tolist()
batch_of_second_sentences = jsonObj['sentence2'].astype(str).values.tolist()

encoded_inputs = tokenizer(batch_sentences, batch_of_second_sentences)

